In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime, math

In [43]:
%%capture
mta = pd.read_csv('origData/mta_1708.csv', error_bad_lines=False)
# mta.set_index('PublishedLineName', inplace=True)
m100 = mta.loc[(mta['PublishedLineName']== 'M100') & (mta['DestinationName'] == 'INWOOD 220 ST via AMSTERDAM via BWAY'),]
arrivals = m100.loc[m100['ArrivalProximityText']=='at stop']
arrivals.head(20)

In [71]:
new = arrivals.loc[:,['RecordedAtTime','VehicleRef']].sort_values(['VehicleRef', 'RecordedAtTime'], ascending=True)
uniqueRef = new['VehicleRef'].unique()
new.RecordedAtTime = pd.to_datetime(new.RecordedAtTime)

## Setting the time deltas only within the same vehicle refs
https://stackoverflow.com/questions/20648346/computing-diffs-within-groups-of-a-dataframe


In [72]:
new.sort_values(['VehicleRef', 'RecordedAtTime'], inplace=True)
new['diffs'] = new['RecordedAtTime'].diff()
mask = new.VehicleRef != new.VehicleRef.shift(1)
new['diffs'][mask] = np.nan
new

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,RecordedAtTime,VehicleRef,diffs
5010109,2017-08-23 10:01:19,NYCT_1202,NaT
5012297,2017-08-23 10:11:48,NYCT_1202,0 days 00:10:29
5044476,2017-08-23 12:51:31,NYCT_1202,0 days 02:39:43
5046521,2017-08-23 13:02:02,NYCT_1202,0 days 00:10:31
5048584,2017-08-23 13:11:27,NYCT_1202,0 days 00:09:25
5057010,2017-08-23 13:51:35,NYCT_1202,0 days 00:40:08
5090596,2017-08-23 16:12:27,NYCT_1202,0 days 02:20:52
5101690,2017-08-23 16:52:25,NYCT_1202,0 days 00:39:58
1394003,2017-08-07 09:33:42,NYCT_1206,NaT
1402955,2017-08-07 10:13:36,NYCT_1206,0 days 00:39:54


In [73]:
new.dropna()
new = new[new.diffs > '02:00:00']
new

,RecordedAtTime,VehicleRef,diffs
5044476,2017-08-23 12:51:31,NYCT_1202,0 days 02:39:43
5090596,2017-08-23 16:12:27,NYCT_1202,0 days 02:20:52
4664821,2017-08-21 20:49:16,NYCT_1215,0 days 03:20:38
3232754,2017-08-15 12:08:01,NYCT_4099,0 days 02:30:17
3281248,2017-08-15 15:47:19,NYCT_4099,0 days 03:39:18
3335690,2017-08-15 19:08:24,NYCT_4099,0 days 03:21:05
3375494,2017-08-15 23:48:27,NYCT_4099,0 days 02:29:48
3664023,2017-08-17 07:41:05,NYCT_4099,1 days 07:32:17
3708439,2017-08-17 10:31:16,NYCT_4099,0 days 02:20:19
3753207,2017-08-17 14:11:42,NYCT_4099,0 days 03:10:34


mta.describe()